# ⚡️ FastPortfolioAnalysis Notebook

Ultra-fast parser for Attijariwafa (and similar) portfolio statements. Downlaod the pdf with your portfolio table from "clients.wafabourse.com", Drop your PDFs into `data/` and run the cells below. The notebook builds an aggregated history (`portfolio_history.csv`) and shows an instant snapshot.



In [ ]:
!pip install pymupdf pandas rich --quiet

import pathlib
from dataclasses import dataclass, asdict, field
from datetime import datetime
import pandas as pd
import fitz  # PyMuPDF
from rich.console import Console
from rich.table import Table
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

import warnings
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [ ]:
# === Configuration ===
RAW_DIR = pathlib.Path('data/')
console = Console()

# === Data Model ===
@dataclass(slots=True, frozen=True)
class Position:
    stock: str
    quantity: float
    price: float
    prmp: float
    statement_date: pd.Timestamp
    amount: float = field(init=False)
    gain: float = field(init=False)
    perf_pct: float = field(init=False)
    
    def __post_init__(self):
        object.__setattr__(self, 'amount', self.quantity * self.price)
        object.__setattr__(self, 'gain', (self.price - self.prmp) * self.quantity)
        object.__setattr__(self, 'perf_pct', (self.price - self.prmp) / self.prmp if self.prmp != 0 else 0)

MONTHS = {
    'janvier':1, 'janv':1, 'fevrier':2, 'février':2, 'fevr':2, 'févr':2,
    'mars':3, 'avril':4, 'avr':4, 'mai':5, 'juin':6,
    'juillet':7, 'juil':7, 'aout':8, 'août':8, 'septembre':9, 'sept':9,
    'octobre':10, 'oct':10, 'novembre':11, 'nov':11, 'decembre':12, 'décembre':12, 'dec':12, 'déc':12
}

def _parse_statement_date(page) -> datetime:
    import re
    txt = page.get_text()
    patterns = [
        r'(\d{1,2})\s+([a-zA-Zéûüçàè\.]+)\s+(\d{4})',
        r'le\s+(\d{1,2})\s+([a-zA-Zéûüçàè\.]+)\s+(\d{4})'
    ]
    for pattern in patterns:
        m = re.search(pattern, txt)
        if m:
            day, month_str, year = m.groups()
            mo = unicodedata.normalize("NFKD", month_str.lower().strip('.'))
            mo = "".join([ch for ch in mo if not unicodedata.combining(ch)])
            month = MONTHS.get(mo, 1)
            return datetime(int(year), month, int(day))
    return datetime.now()

def safe_float_convert(value_str):
    if pd.isna(value_str) or value_str is None:
        return None
    clean_str = str(value_str).strip().replace(',', '.').replace(' ', '').replace('\xa0', '')
    if '%' in clean_str:
        clean_str = clean_str.replace('%', '')
        try:
            return float(clean_str) / 100
        except ValueError:
            return None
    try:
        return float(clean_str)
    except (ValueError, TypeError):
        return None

def parse_portfolio_pdfs(raw_dir: pathlib.Path):
    all_positions = []
    for pdf in raw_dir.glob('*.pdf'):
        console.print(f'[cyan]Processing {pdf.name}...[/cyan]')
        doc = fitz.open(pdf)
        for page in doc:
            tables = page.find_tables()
            if not tables:
                continue
            stmt_date = _parse_statement_date(page)
            for t in tables:
                df = t.to_pandas()
                if df.shape[1] < 7:
                    continue
                for idx, row in df.iterrows():
                    stock_val = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else ""
                    if stock_val.lower() in ['valeur', 'totaux'] or len(stock_val) < 2:
                        continue
                    qty_raw, price_raw, prmp_raw = row.iloc[2], row.iloc[3], row.iloc[6]
                    quantity = safe_float_convert(qty_raw)
                    price = safe_float_convert(price_raw)
                    prmp = safe_float_convert(prmp_raw)
                    if all(x is not None and x > 0 for x in [quantity, price, prmp]):
                        all_positions.append(Position(
                            stock=stock_val, quantity=quantity, price=price,
                            prmp=prmp, statement_date=pd.Timestamp(stmt_date)
                        ))
    return pd.DataFrame([asdict(p) for p in all_positions])

# Parse and display
df = parse_portfolio_pdfs(RAW_DIR)
df_sorted = df.sort_values('amount', ascending=False)
df_sorted.to_csv("portfolio.csv", index=False)

# Portfolio Summary Table with symbols for gains/losses 
portfolio_df = pd.DataFrame({
    "Stock": [row['stock'][:19] for _, row in df_sorted.iterrows()],
    "Qty": [f"{row['quantity']:.0f}" for _, row in df_sorted.iterrows()],
    "Price": [f"{row['price']:.2f}" for _, row in df_sorted.iterrows()],
    "Amount": [f"{row['amount']:,.0f}" for _, row in df_sorted.iterrows()],
    "PRMP": [f"{row['prmp']:.2f}" for _, row in df_sorted.iterrows()],
    "Gain/Loss": [f"{'🟢' if row['gain'] >= 0 else '🔴'} {row['gain']:+,.0f}" for _, row in df_sorted.iterrows()],
    "Perf %": [f"{'🟢' if row['perf_pct'] >= 0 else '🔴'} {row['perf_pct']*100:+.1f}%" for _, row in df_sorted.iterrows()]
})
total_amount, total_gain = df_sorted['amount'].sum(), df_sorted['gain'].sum()

display(portfolio_df.style.hide(axis="index").set_properties(**{
    'font-weight': 'bold', 'color': 'black', 'background-color': 'white', 'text-align': 'center'
}).set_table_styles([
    {'selector': 'th', 'props': [('font-weight', 'bold'), ('color', 'black'), ('background-color', 'lightgray'), ('text-align', 'center')]}
]))

print(f"📊 Portfolio - {df['statement_date'].iloc[0].strftime('%d %b %Y')}")
print(f"📊 {len(df)} positions | {total_amount:,.0f} MAD | P&L: {total_gain:+,.0f} MAD\n")


In [ ]:
# Set modern style
plt.style.use('default')
sns.set_theme(style="whitegrid", palette="husl")

# Calculate additional metrics
df_viz = df_sorted.copy()
df_viz['weight'] = df_viz['amount'] / df_viz['amount'].sum()
df_viz['abs_gain'] = df_viz['gain'].abs()

print("🎨 Generating portfolio visualizations...")

# 1. 🍰 PORTFOLIO ALLOCATION PIE CHART - ALL POSITIONS
fig = go.Figure(data=[go.Pie(
    labels=[stock[:15] for stock in df_viz['stock']],  # ALL positions, truncate long names
    values=df_viz['weight'],
    hole=0.4,
    textinfo='label+percent',
    textposition='auto',
    marker=dict(colors=px.colors.qualitative.Set3 + px.colors.qualitative.Pastel),  # More colors
    hovertemplate='<b>%{label}</b><br>Weight: %{percent}<br>Amount: %{customdata:,.0f} MAD<extra></extra>',
    customdata=df_viz['amount']
)])
fig.update_layout(
    title="🍰 Portfolio Allocation (All Holdings)",
    font=dict(size=10),  # Smaller font for more labels
    height=600,  # Taller to accommodate all labels
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.05
    )
)
fig.show()

# 2. 📊 PERFORMANCE WATERFALL CHART
fig = go.Figure(go.Waterfall(
    name="Performance", 
    orientation="v",
    measure=["relative"] * len(df_viz) + ["total"],
    x=[stock[:10] for stock in df_viz['stock']] + ["Total P&L"],
    y=list(df_viz['gain']) + [df_viz['gain'].sum()],
    text=[f"{x:+,.0f}" for x in df_viz['gain']] + [f"{df_viz['gain'].sum():+,.0f}"],
    connector={"line": {"color": "rgb(63, 63, 63)"}},
    increasing={"marker": {"color": "green"}},
    decreasing={"marker": {"color": "red"}},
    totals={"marker": {"color": "blue"}}
))
fig.update_layout(
    title="💰 Portfolio P&L Waterfall Analysis",
    xaxis_tickangle=-45,
    height=600,
    xaxis_title="Stocks",
    yaxis_title="Gain/Loss (MAD)"
)
fig.show()

# 3. 📈 TREEMAP VISUALIZATION  
fig = px.treemap(
    df_viz, 
    path=[px.Constant("Portfolio"), 'stock'], 
    values='amount',
    color='perf_pct',
    color_continuous_scale='RdYlGn',
    color_continuous_midpoint=0,
    title="🗺️ Portfolio Treemap (Size=Amount, Color=Performance)",
    hover_data=['quantity', 'price', 'prmp']
)
fig.update_layout(height=600)
fig.show()


# 4. 🎨 INTERACTIVE DASHBOARD SUMMARY
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('📈 Performance Distribution', '💰 Amount vs Gain', 
                   '⚖️ Weight Distribution', '🎯 Gain per MAD Invested'),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Performance histogram
fig.add_trace(
    go.Histogram(x=df_viz['perf_pct']*100, name="Performance %", nbinsx=15,
                marker_color='lightblue', opacity=0.7), 
    row=1, col=1
)

# Amount vs Gain scatter
fig.add_trace(go.Scatter(
    x=df_viz['amount'], y=df_viz['gain'],
    mode='markers', 
    text=[f"{stock}<br>Perf: {perf*100:+.1f}%" for stock, perf in zip(df_viz['stock'], df_viz['perf_pct'])],
    marker=dict(
        size=10, 
        color=df_viz['perf_pct'], 
        colorscale='RdYlGn',
        line=dict(width=1, color='black')
    ),
    name="Amount vs Gain",
    hovertemplate='<b>%{text}</b><br>Amount: %{x:,.0f}<br>Gain: %{y:+,.0f}<extra></extra>'
), row=1, col=2)

# Weight distribution (ALL positions)
fig.add_trace(go.Bar(
    x=[stock[:8] for stock in df_viz['stock']], 
    y=df_viz['weight']*100,
    name="Weight %", 
    marker_color='lightcoral',
    text=[f"{w:.1f}%" for w in df_viz['weight']*100],
    textposition='outside'
), row=2, col=1)

# Efficiency (Gain per MAD invested) - ALL positions
efficiency = (df_viz['gain'] / df_viz['amount']) * 100
fig.add_trace(go.Bar(
    x=[stock[:8] for stock in df_viz['stock']], 
    y=efficiency,
    name="Efficiency %", 
    marker_color='gold',
    text=[f"{e:+.1f}%" for e in efficiency],
    textposition='outside'
), row=2, col=2)

fig.update_layout(
    height=800, 
    title_text="🎨 Portfolio Dashboard Overview",
    showlegend=False
)
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_xaxes(tickangle=45, row=2, col=2)
fig.show()

# 5. 🏆 TOP WINNERS & LOSERS
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))  # Made much wider and taller

# Winners - ALL positive performers
winners = df_viz[df_viz['perf_pct'] > 0].sort_values('perf_pct', ascending=True)
if not winners.empty:
    y_pos = np.arange(len(winners))
    bars1 = ax1.barh(y_pos, winners['perf_pct'] * 100, color='#2E8B57', alpha=0.8)
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels([stock[:15] for stock in winners['stock']], fontsize=11)
    ax1.set_title('🏆 Top Winners', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Performance (%)', fontsize=12)
    
    # Smart label placement for small and large bars
    for i, (_, row) in enumerate(winners.iterrows()):
        bar_width = row['perf_pct'] * 100
        percentage_text = f"{row['perf_pct']*100:.1f}%"
        
        if bar_width < 5:  # For very small bars, place text outside
            ax1.text(bar_width + 0.5, i, percentage_text, 
                    va='center', ha='left', fontsize=10, fontweight='bold', color='black')
        elif bar_width < 15:  # For small bars, place text at the end
            ax1.text(bar_width - 0.5, i, percentage_text, 
                    va='center', ha='right', fontsize=10, fontweight='bold', color='white')
        else:  # For larger bars, place text in the center
            ax1.text(bar_width / 2, i, percentage_text, 
                    va='center', ha='center', fontsize=10, fontweight='bold', color='white')
    
    # Extend x-axis slightly to accommodate outside labels
    ax1.set_xlim(0, max(winners['perf_pct'] * 100) * 1.1)

# Losers - ALL negative performers
losers = df_viz[df_viz['perf_pct'] < 0].sort_values('perf_pct', ascending=False)
if not losers.empty:
    y_pos = np.arange(len(losers))
    bars2 = ax2.barh(y_pos, losers['perf_pct'] * 100, color='#DC143C', alpha=0.8)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([stock[:15] for stock in losers['stock']], fontsize=11)
    ax2.set_title('📉 Underperformers', fontsize=16, fontweight='bold')
    ax2.set_xlabel('Performance (%)', fontsize=12)
    
    # Smart label placement for negative bars
    for i, (_, row) in enumerate(losers.iterrows()):
        bar_width = row['perf_pct'] * 100  # This will be negative
        percentage_text = f"{row['perf_pct']*100:.1f}%"
        
        if abs(bar_width) < 5:  # For very small bars, place text outside (to the left)
            ax2.text(bar_width - 0.5, i, percentage_text, 
                    va='center', ha='right', fontsize=10, fontweight='bold', color='black')
        elif abs(bar_width) < 15:  # For small bars, place text at the end
            ax2.text(bar_width + 0.5, i, percentage_text, 
                    va='center', ha='left', fontsize=10, fontweight='bold', color='white')
        else:  # For larger bars, place text in the center
            ax2.text(bar_width / 2, i, percentage_text, 
                    va='center', ha='center', fontsize=10, fontweight='bold', color='white')
    
    # Extend x-axis slightly to accommodate outside labels
    ax2.set_xlim(min(losers['perf_pct'] * 100) * 1.1, 0)
else:
    ax2.text(0.5, 0.5, '🎉 No Losing Positions!', ha='center', va='center',
             transform=ax2.transAxes, fontsize=16, color='green', fontweight='bold')

# Add grid for better readability
ax1.grid(True, alpha=0.3, axis='x')
if not losers.empty:
    ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# 6. 🎯 RISK-RETURN SCATTER PLOT
fig, ax = plt.subplots(figsize=(12, 8))
colors = ['#2E8B57' if x > 0 else '#DC143C' for x in df_viz['perf_pct']]
sizes = df_viz['amount'] / 50

scatter = ax.scatter(
    df_viz['weight'] * 100, 
    df_viz['perf_pct'] * 100,
    s=sizes,
    c=colors, 
    alpha=0.7, 
    edgecolors='black',
    linewidth=0.5
)

ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.set_xlabel('Portfolio Weight (%)', fontsize=12)
ax.set_ylabel('Performance (%)', fontsize=12)
ax.set_title('🎯 Risk-Return Analysis\n(Bubble size = Investment amount)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add annotations for top 5 holdings
for _, row in df_viz.head(5).iterrows():
    ax.annotate(row['stock'][:8], 
                (row['weight']*100, row['perf_pct']*100),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, alpha=0.8,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.7))

plt.tight_layout()
plt.show()


# 7. 📊 PORTFOLIO COMPOSITION BY VALUE RANGES
def categorize_holding(amount):
    if amount >= 3000: return "🔥 Large (≥3K)"
    elif amount >= 1000: return "🟡 Medium (1K-3K)"
    else: return "🔵 Small (<1K)"

df_viz['size_category'] = df_viz['amount'].apply(categorize_holding)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# By value
sizes = df_viz.groupby('size_category')['amount'].sum()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
wedges1, texts1, autotexts1 = ax1.pie(sizes.values, labels=sizes.index, autopct='%1.1f%%', 
                                      startangle=90, colors=colors)
ax1.set_title('📊 Holdings by Value Distribution', fontsize=12, fontweight='bold')

# By count
counts = df_viz.groupby('size_category').size()
wedges2, texts2, autotexts2 = ax2.pie(counts.values, labels=counts.index, 
                                      autopct='%1.0f stocks', startangle=90, colors=colors)
ax2.set_title('📊 Holdings by Count Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# 8. 📊 PORTFOLIO ANALYTICS SUMMARY TABLE
total_invested = (df_viz['amount'] - df_viz['gain']).sum()
roi = (df_viz['gain'].sum() / total_invested) * 100
winning_positions = len(df_viz[df_viz['gain'] > 0])
losing_positions = len(df_viz[df_viz['gain'] < 0])

metrics_df = pd.DataFrame({
    "Metric": [
        "🎯 Total Positions", "💰 Current Value", "📈 Total Invested", 
        "💵 Unrealized P&L", "📊 ROI", "🏆 Winning Positions",
        "📉 Losing Positions", "🥇 Best Performer", "🥲 Worst Performer",
        "⚖️ Portfolio Concentration"
    ],
    "Value": [
        len(df_viz), f"{df_viz['amount'].sum():,.0f} MAD", f"{total_invested:,.0f} MAD",
        f"{df_viz['gain'].sum():+,.0f} MAD", f"{roi:+.1f}%", 
        f"{winning_positions}/{len(df_viz)} ({winning_positions/len(df_viz)*100:.0f}%)",
        f"{losing_positions}/{len(df_viz)} ({losing_positions/len(df_viz)*100:.0f}%)",
        f"{df_viz.loc[df_viz['perf_pct'].idxmax(), 'stock']} ({df_viz['perf_pct'].max()*100:+.1f}%)",
        f"{df_viz.loc[df_viz['perf_pct'].idxmin(), 'stock']} ({df_viz['perf_pct'].min()*100:+.1f}%)",
        f"Top 5 = {df_viz['weight'].head(5).sum()*100:.1f}%"
    ]
})

display(metrics_df.style.hide(axis="index").set_properties(**{
    'font-weight': 'bold', 'color': 'black', 'background-color': 'white'
}))